# Model 03 inference

Evidence retrieval using a siamese BERT classification model. This is similar to Model 01 except this does not use any community based models from sentence transformer.

On additional training using increased negative samples.

## Setup

### Working directory

In [1]:
# Change the working directory to project root
from pathlib import Path
import os
ROOT_DIR = Path.cwd()
while not ROOT_DIR.joinpath("src").exists():
    ROOT_DIR = ROOT_DIR.parent
os.chdir(ROOT_DIR)

### File paths

In [2]:
MODEL_PATH = ROOT_DIR.joinpath("./result/models/*")
OUTPUT_PATH = ROOT_DIR.joinpath("./result/inference/*")
DATA_PATH = ROOT_DIR.joinpath("./data/*")
NER_PATH = ROOT_DIR.joinpath("./result/ner/*")

### Dependencies

In [3]:
# Imports and dependencies
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import Linear, Module, CrossEntropyLoss, Dropout, CosineSimilarity
from transformers import BertModel, BertTokenizer
from torch.optim import Adam
from torch.optim.lr_scheduler import LinearLR
from torch.nn.functional import relu, softmax
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryRecall

from src.torch_utils import get_torch_device
from src.data import create_claim_output
import json
from dataclasses import dataclass
from typing import List, Union, Tuple, Dict
from tqdm import tqdm
import random
import numpy as np
from datetime import datetime
from math import exp
from collections import defaultdict

TORCH_DEVICE = get_torch_device()

/opt/homebrew/Caskroom/miniconda/base/envs/comp90042_project/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch device is 'mps'


## Dataset

In [4]:
@dataclass
class ClaimEvidencePair:
    claim_id:str
    evidence_id:str
    label:int = 0

In [5]:
class InferenceClaims(Dataset):
    
    def __init__(self, claims_path:Path) -> None:
        super(InferenceClaims, self).__init__()
        with open(claims_path, mode="r") as f:
            self.claims = json.load(fp=f)
            self.claim_ids = list(self.claims.keys())
            print(f"loaded inference claims n={len(self.claim_ids)}")
        return
        
    def __len__(self):
        return len(self.claim_ids)

    def __getitem__(self, idx) -> Tuple[str]:
        claim_id = self.claim_ids[idx]
        claim_text = self.claims[claim_id]["claim_text"]
        return claim_id, claim_text

In [6]:
class SiameseDatasetInference(Dataset):
    
    def __init__(
        self,
        claim_id:str,
        claims_path:Path,
        evidence_path:Path,
        claims_shortlist:Path = None,
        evidence_shortlist:Path = None,
        verbose:bool=True
    ) -> None:
        super(SiameseDatasetInference, self).__init__()
        self.verbose = verbose
        self.claim_id = claim_id
        
        # Load claims text from json
        with open(claims_path, mode="r") as f:
            claims = json.load(fp=f)
            self.claim_text = claims[self.claim_id]["claim_text"]
            
        # Load evidence library
        with open(evidence_path, mode="r") as f:
            self.evidence = json.load(fp=f)
        
        # Load the pre-retrieved shortlist of evidences
        # From either a pre-retrieved list of evidences specific for the
        # claim_id or from a pre-collated evidence shortlist
        # Both of which were determined from the fast shortlisting process
        if claims_shortlist is not None:
            with open(claims_shortlist, mode="r") as f:
                claims_shortlist_ = json.load(fp=f)
                self.evidence_shortlist = list(set(
                    claims_shortlist_.get(self.claim_id, [])
                ))
                print(f"loaded claims_shortlist: {claims_shortlist}")
        elif evidence_shortlist is not None:
            with open(evidence_shortlist, mode="r") as f:
                self.evidence_shortlist = list(set(json.load(fp=f)))
                print(f"loaded evidence_shortlist: {evidence_shortlist}")
        else:
            raise Exception(
                "Provide either a claims_shortlist or evidence_shortlist"
            )

        return

    def __len__(self):
        return len(self.evidence_shortlist)
    
    def __getitem__(self, idx) -> Tuple[str]:
        # Get text ids
        claim_id = self.claim_id
        evidence_id = self.evidence_shortlist[idx]
        
        # Get text
        claim_text = self.claim_text
        evidence_text = self.evidence[evidence_id]

        return (claim_text, evidence_text, claim_id, evidence_id)

## Load model

In [7]:
class SiameseClassifierBert(Module):
    
    def __init__(
            self,
            pretrained_name:str,
            device,
            **kwargs
        ) -> None:
        super(SiameseClassifierBert, self).__init__(**kwargs)
        self.device = device
        
        # Use a pretrained tokenizer
        self.tokenizer = BertTokenizer.from_pretrained(pretrained_name)
        
        # Use a pretrained model
        self.bert = BertModel.from_pretrained(pretrained_name)
        self.bert.to(device=device)
        
        # Classification layers
        self.linear1 = Linear(2304, 1024, bias=True, device=device)
        self.linear2 = Linear(1024, 512, bias=True, device=device)
        self.linear3 = Linear(512, 2, bias=True, device=device)
        self.relu = relu
        self.softmax = softmax
        self.dropout_in = Dropout(p=0.2)
        self.dropout_out = Dropout(p=0.5)
        
        # print(self.tokenizer)
        # print(self.bert)
        # print(self.linear1)
        # print(self.linear2)
        # print(self.activation)
        # print(self.softmax)
        return
        
    def forward(self, claim_texts, evidence_texts) -> Tuple[torch.Tensor]:
        
        # Run the tokenizer
        t_kwargs = {
            "return_tensors": "pt",
            "padding": True,
            "truncation": True,
            "max_length": 100,
            "add_special_tokens":True
        }
        claim_x = self.tokenizer(claim_texts, **t_kwargs)
        evidence_x = self.tokenizer(evidence_texts, **t_kwargs)
        
        claim_x = claim_x["input_ids"].to(device=self.device)
        evidence_x = evidence_x["input_ids"].to(device=self.device)
        
        # Run Bert
        claim_x = self.bert(claim_x, return_dict=True).pooler_output
        evidence_x = self.bert(evidence_x, return_dict=True).pooler_output
        # dim=768
        
        # Concatenate the two embeddings
        x = torch.cat((claim_x, evidence_x, claim_x - evidence_x), dim=1)
        # dim=2304
        
        # Run classification layers
        x = self.dropout_in(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout_out(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.dropout_out(x)
        x = self.linear3(x)
        
        # Create the predictions
        y = self.softmax(x, dim=-1)
        
        return (y, claim_x, evidence_x)

In [8]:
MODEL_NAME = f"model_03_base_run_01_continue.pth"
BATCH_SIZE = 64

In [9]:
with open(MODEL_PATH.with_name(MODEL_NAME), mode="rb") as f:
    model = torch.load(f, map_location=TORCH_DEVICE)

## Inference run code

In [10]:
@dataclass
class EvidenceScore:
    evidence_id:str
    score:float
    
    def to_list(self) -> List[str]:
        return [self.evidence_id, str(self.score)]
    
    def to_dict(self) -> Dict[str, str]:
        return {"evidence_id": self.evidence_id, "score": str(self.score)}

In [11]:
def inference_run(
    claims_path:Path,
    evidence_path:Path,
    claims_shortlist:Path = None,
    evidence_shortlist:Path = None,
    save_path:Path = None,
    batch_size:int = 64
):
    # Generate claims iterations
    inference_claims = InferenceClaims(claims_path=claims_path)
    
    # Cumulator
    claim_predictions = dict()
    
    # Load from save to continue inference if exists
    saved_predictions = dict()
    if save_path.exists():
        with open(save_path, mode="r") as f:
            saved_predictions.update(json.load(fp=f))
    
    for claim_id, claim_text in inference_claims:
        
        # Skip if a save entry exists
        if claim_id in saved_predictions.keys():
            print(f"skipping {claim_id}, done previously")
            continue
        
        # Create a save entry
        claim_predictions[claim_id] = {
            "claim_text": claim_text,
            "evidences": []
        }
        
        # Generate claims-evidence inference interations
        infer_data = SiameseDatasetInference(
            claim_id=claim_id,
            claims_path=claims_path,
            evidence_path=evidence_path,
            claims_shortlist=claims_shortlist,
            evidence_shortlist=evidence_shortlist,
        )
        infer_dataloader = DataLoader(
            dataset=infer_data,
            shuffle=False,
            batch_size=batch_size
        )
        
        print(f"running inference for {claim_id} n={len(infer_data)}")
    
        # Set model mode to evaluation
        model.eval()
        
        infer_batches = tqdm(infer_dataloader, desc="infer batches")
        for batch in infer_batches:
            claim_texts, evidence_texts, batch_claim_ids, batch_evidence_ids = batch
            
            # Forward
            predictions, claim_emb, evidence_emb = model(claim_texts, evidence_texts)
            
            # Prediction
            _, predicted_labels = torch.max(predictions, dim=-1)
            
            # Score embeddings with cosine similarity
            cos_sim = CosineSimilarity(dim=1)
            scores = cos_sim(claim_emb, evidence_emb)
            
            for c_id, e_id, predicted_label, score in zip(
                batch_claim_ids, batch_evidence_ids,
                predicted_labels.cpu().numpy(),
                scores.cpu().detach().numpy()
            ):
                if predicted_label == 1:
                    claim_predictions[c_id]["evidences"].append(EvidenceScore(
                        evidence_id=e_id,
                        score=score
                    ))
            
            continue
    
        # Save on every claim_id
        if save_path:
            # Retrieve at most 5 top predicted evidences by score
            claim_predictions_output = dict()
            for claim_id_, claim_ in claim_predictions.items():
                claim_ = claim_.copy()
                claim_["evidences"] = [
                    evidence_score.evidence_id
                    for evidence_score in sorted(
                        claim_["evidences"],
                        key=lambda es: es.score,
                        reverse=True
                    )
                ][:5]
                claim_predictions_output[claim_id_] = claim_
            
            # Make a copy of existing saved results and add the new results
            # for this run
            save_dict = saved_predictions.copy()
            save_dict.update(claim_predictions_output)

            with open(save_path, mode="w") as f:
                json.dump(obj=save_dict, fp=f)
                print(f"saved to: {save_path}")
    
        continue
    return

## Dev inference

In [12]:
inference_run(
    claims_path=DATA_PATH.with_name("dev-claims.json"),
    evidence_path=DATA_PATH.with_name("evidence.json"),
    claims_shortlist=NER_PATH.with_name("dev_claim_evidence_retrieved.json"),
    # evidence_shortlist=NER_PATH.with_name("shortlist_dev_claim_evidence_retrieved.json"),
    save_path=OUTPUT_PATH.with_name("model_03_run_01_continue_dev_claim_evidence_retrieved.json")
)

loaded inference claims n=154
loaded claims_shortlist: /Users/johnsonzhou/git/comp90042-project/result/ner/dev_claim_evidence_retrieved.json
running inference for claim-752 n=1564


infer batches: 100%|██████████| 25/25 [00:05<00:00,  4.21it/s]


saved to: /Users/johnsonzhou/git/comp90042-project/result/inference/model_03_run_01_continue_dev_claim_evidence_retrieved.json
loaded claims_shortlist: /Users/johnsonzhou/git/comp90042-project/result/ner/dev_claim_evidence_retrieved.json
running inference for claim-375 n=3682


infer batches: 100%|██████████| 58/58 [00:15<00:00,  3.75it/s]


saved to: /Users/johnsonzhou/git/comp90042-project/result/inference/model_03_run_01_continue_dev_claim_evidence_retrieved.json
loaded claims_shortlist: /Users/johnsonzhou/git/comp90042-project/result/ner/dev_claim_evidence_retrieved.json
running inference for claim-1266 n=0


...

saved to: /Users/johnsonzhou/git/comp90042-project/result/inference/model_03_run_01_continue_dev_claim_evidence_retrieved.json
loaded claims_shortlist: /Users/johnsonzhou/git/comp90042-project/result/ner/dev_claim_evidence_retrieved.json
running inference for claim-698 n=3627


infer batches: 100%|██████████| 57/57 [00:17<00:00,  3.30it/s]


saved to: /Users/johnsonzhou/git/comp90042-project/result/inference/model_03_run_01_continue_dev_claim_evidence_retrieved.json
loaded claims_shortlist: /Users/johnsonzhou/git/comp90042-project/result/ner/dev_claim_evidence_retrieved.json
running inference for claim-1021 n=2017


infer batches: 100%|██████████| 32/32 [00:08<00:00,  3.82it/s]

saved to: /Users/johnsonzhou/git/comp90042-project/result/inference/model_03_run_01_continue_dev_claim_evidence_retrieved.json


## Test inference

In [13]:
# inference_run(
#     claims_path=DATA_PATH.with_name("test-claims-unlabelled.json"),
#     evidence_path=DATA_PATH.with_name("evidence.json"),
#     # claims_shortlist=NER_PATH.with_name("test_claim_evidence_retrieved.json"),
#     evidence_shortlist=NER_PATH.with_name("shortlist_test_claim_evidence_retrieved.json"),
#     save_path=OUTPUT_PATH.with_name("model_03_run_01_test_claim_evidence_retrieved.json")
# )